In [ ]:
# Helper Box Filters
import numpy as np
import copy
import matplotlib.pyplot as plt

class boxFilters:
    def __init__(self,ogFilterSize):
        self.alpha = None
        self.a=None
        self.b=None
        self.c=None
        self.d=None
        self.ogFilterSize = ogFilterSize
    
    def setAlpha(self,alpha):
        self.alpha = alpha
    
    def setABCD(self,a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
    
    def printParams(self):
        print("A:",self.a,"B:",self.b,"C:",self.c,"D:",self.d, "Alpha:", self.alpha)


def overlapping_area(box1, box2):
    # Extract coordinates of the top-left and bottom-right corners of each box
    x1_tl, y1_tl, x1_br, y1_br = box1.a, box1.c, box1.b, box1.d
    x2_tl, y2_tl, x2_br, y2_br = box2.a, box2.c, box2.b, box2.d
    #print(x1_tl, y1_tl, x1_br, y1_br)
    #print(x2_tl, y2_tl, x2_br, y2_br)
    # Calculate coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    y_intersection_tl = max(y1_tl, y2_tl)
    x_intersection_br = min(x1_br, x2_br)
    y_intersection_br = min(y1_br, y2_br)
    

    # Check if there is an actual intersection
    if x_intersection_tl <= x_intersection_br and y_intersection_tl <= y_intersection_br:
        # Calculate the width and height of the intersection rectangle
        intersection_width = x_intersection_br - x_intersection_tl + 1
        intersection_height = y_intersection_br - y_intersection_tl + 1
        # Calculate the area of the intersection rectangle
       # print(intersection_width,intersection_height)
        intersection_area = intersection_width * intersection_height
        return intersection_area
    else:
        # No intersection
        return 0

def sum_within_boundary(filter, x1, y1, x2, y2):
    # Ensure x1 <= x2 and y1 <= y2
    x1, x2 = min(x1, x2), max(x1, x2)
    y1, y2 = min(y1, y2), max(y1, y2)

    # Ensure the boundary coordinates are within the matrix dimensions
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(len(filter) - 1, x2), min(len(filter[0]) - 1, y2)
   # print("x1:",x1,"y1:",y1,"x2:",x2,"y2:",y2)
    # Calculate the sum within the boundary
    boundary_sum = 0
    for i in range(y1, y2 + 1):
        for j in range(x1, x2 + 1):
            #print(i,j)
            boundary_sum += filter[i][j]
           # print(boundary_sum)
    return boundary_sum



def calcAlpha(boxFiltersList,filter):

    A = np.ones((len(boxFiltersList),len(boxFiltersList)))
    for i in range(0,len(boxFiltersList)):
        for j in range(0,len(boxFiltersList)):
           A[j][i]  =  overlapping_area(boxFiltersList[i],boxFiltersList[j])    
    
    det = np.linalg.det(A)
    #print("A:",A)
    AInv = None
    if np.isclose(det, 0):
        #print("Singular")
        pass
        
    else:
        # Calculate the inverse
        AInv = np.linalg.inv(A)
    
    B = np.ones((len(boxFiltersList)))

    for i in range(0,len(boxFiltersList)):
        B[i] = sum_within_boundary(filter,boxFiltersList[i].a,boxFiltersList[i].c,boxFiltersList[i].b,boxFiltersList[i].d)
    #print("B:",B)
    if(np.all(AInv==None)):
        return None
    else:
     return np.matmul(AInv,B)    
    

def l2_distance(arr1, arr2):
    # Flatten the arrays to treat them as vectors
    vec1 = arr1.flatten()
    vec2 = arr2.flatten()
    
    # Calculate the L2 distance
    distance = np.linalg.norm(vec1 - vec2)
    
    return distance

def percentage_l2_error(matrix1, matrix2):
    # Calculate the L2 norm of the difference between the matrices
    diff_norm = np.linalg.norm(matrix1 - matrix2)
    #print(diff_norm)
    # Calculate the L2 norm of one of the matrices
    norm_matrix1 = np.linalg.norm(matrix1)

    # Calculate the percentage L2 error
    percent_error = (diff_norm / norm_matrix1) * 100

    return percent_error



def computeTempFilter(tempBoxFilterList):
    tempFilter = np.zeros((tempBoxFilterList[0].ogFilterSize,tempBoxFilterList[0].ogFilterSize))
    for i in range(0,len(tempBoxFilterList)):
        for j in range(tempBoxFilterList[i].c,tempBoxFilterList[i].d+1):
            for k in range(tempBoxFilterList[i].a,tempBoxFilterList[i].b+1):
                #print(j,k)
                tempFilter[j][k] = tempFilter[j][k] + tempBoxFilterList[i].alpha

    return tempFilter
    



def calculateBoxFilters(filter,maxN):

    N=0
    maxN = maxN
    boundaries = []
    boxFilterList = []
    maxSize = filter.shape[0]
    aMax = maxSize
    bMax = maxSize
    cMax = maxSize
    dMax = maxSize




    boxFilterList = []
    minLoss = None
    minLossBox = None
    minLossBoxes = []
    flag = True
    while(N<maxN):
        
    
        #print("*****************************************************")
        #print("N:",N)
        for a in range(0,aMax):
            for b in range(0,bMax):
                for c in range(0,cMax):
                    for d in range(0,dMax):
                        #print("ABCD:",a,b,c,d)
                        tempBox = boxFilters(maxSize)
                        tempBox.setABCD(a,b,c,d)
                        #tempBox.printParams()
                        # print("tempBoxFilterList: Before")
                        # for i in tempBoxFilterList:
                        #  i.printParams() 
                        tempBoxFilterList = copy.deepcopy(boxFilterList)
                        tempBoxFilterList.append(tempBox)
                        #print("tempBoxFilterList: After")
                        # for i in tempBoxFilterList:
                        #     i.printParams() 

                        tempAlpha = calcAlpha(tempBoxFilterList,filter)
                        
                        if(np.all(tempAlpha)!=None):
                        #print("Alpha: ",tempAlpha)
                         for i,alpha in enumerate(tempAlpha):
                            
                            tempBoxFilterList[i].setAlpha(alpha)
                            #print(tempBoxFilterList[i].alpha)
                        #tempBox.setAlpha(tempAlpha[-1])
                        #print("tempBoxFilterList: After Alpha")
                        #  for i in tempBoxFilterList:
                        #   i.printParams() 
                         tempFilter = computeTempFilter(tempBoxFilterList)
                         loss = percentage_l2_error(tempFilter,filter)
                        #print("Loss:",loss)
                         if(minLoss==None):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         elif(loss<minLoss):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         if(np.isclose(minLoss,0)):
                            flag = False
                         if(not flag):
                            break
                    if(not flag):
                        break
                if(not flag):
                    break
            if(not flag):
                break
        if(not flag):
            break


        #print("-------------------------")
        print(N,"   ", minLoss)  
        if(minLossBox!=None):
         #print(minLoss,minLossBox.alpha,N)

         boxFilterList.append(minLossBox)
        N = N +1
    return minLossBoxes,minLoss                   

        

In [ ]:

filter = np.array([[1,1,1],[-2,-2,0],[-2,3,-3]])


boxFilterList,loss = calculateBoxFilters(filter,5)
print(loss)